In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

from IPython.display import Markdown, display
from IPython.display import clear_output

from itertools import combinations
import pandas as pd
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import os
import re

C:\Users\justin\AppData\Local\Programs\Python\Python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
C:\Users\justin\AppData\Local\Programs\Python\Python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


In [2]:
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")

In [2]:
nlp_ministries = spacy.load("..\\data\\spacy labeled\\output\\model-last")
df = pd.read_csv('..\\data\\ocred\\files_df.csv', index_col = 0)

# dates
This first part will be the evalutation for the base dates extractor

In [49]:
months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december',
         'january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december',
         'jan', 'feb', 'mrt', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
sent = ['datum', 'verzonden', 'sent', 'date', 'received']

In [172]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')
    
def showMatches(doc, matches, regexMatches):   
    indexOfMatches = []
    for matchid, start, end in matches:
        for i in range(start, end):
            indexOfMatches.append(i)
            
    indexOfMatches = set(indexOfMatches)
    
    for token in doc:
        if token.i in indexOfMatches:
            printHilight(str(token.text))
        else:
            if token.text in regexMatches:
                printHilight(str(token.text))
            else:
                print(token, end=' ')
    
    return
    
    

In [151]:
def validate(dates, sep, pat):
    goodDates = []
    for date in dates:
        date = date.replace(' ', '')
        try:
            date = date.replace(sep, ' ')
            datetime.strptime(date, pat)
            goodDates.append(date.replace(' ', sep))
        except:
            try:
                if len(date.split(' ')) == 3 and len(date.split(' ')[2]) == 2:
                    datetime.strptime(date, '%d %m %y')
                    goodDates.append(date.replace(' ', sep))
            except:
                pass     
    return goodDates
            

def regexMatcher(text):
    results = []
    
    results += validate(re.findall('[0-3]{0,1}[0-9]\/[0-1]{0,1}[0-9]', text), '/', '%d %m')

    results += validate(re.findall('[0-3]{0,1}[0-9]\/[0-1]{0,1}[0-9]\/[0-9]{2,4}', text), '/', '%d %m %Y')

    results += validate(re.findall('[0-3]{0,1}[0-9]-[0-1]{0,1}[0-9]', text), '-', '%d %m')

    results += validate(re.findall('[0-3]{0,1}[0-9]-[0-1]{0,1}[0-9]-[0-9]{2,4}', text), '-', '%d %m %Y')
        
    return results

In [170]:
def inputHandling(message):
    while(True):
        i = input(message)
        if i == 'exit':
            return -1
        
        elif i == '':
            return 0
        
        try:
            i = int(i)
            return i
        
        except:
            print("input number, exit or nothing")
        

def evaluate(nlp):
    
    months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december',
         'january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december',
         'jan', 'feb', 'mrt', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag',
           'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
           'ma', 'di', 'wo', 'woe', 'do', 'vrij', 'za', 'zat', 'zo', 'vr']
    
    datesPattern = [{"LOWER" : {"IN" : days}, "OP" : "?"}, 
           {"IS_DIGIT": True}, 
           {"LOWER" : {"IN" : months}},
           {"IS_PUNCT" : True, "OP" : "?", "TEXT":'.'},
           {"IS_DIGIT": True, "OP" : "?"}]
    matcher = Matcher(nlp.vocab)
    matcher.add("Dates", [datesPattern])
    
    # cor = exact match, inc = match is wrong (wrong bounds or label), mis = missing match, spu = found something that isnt a mactch
    results = {
        'correct':0,
        'incorrect':0,
        'missing':0,
        'spurious':0
    }
    
    
    while(True):
        try:
            print(sum(results.values()))
            sample = df.sample(1)
            print(sample.name.values[0], sample.page.values[0], '\n\n')
            text = sample.text.values[0]
            text = re.sub('\n+', '\n', text)
            text = re.sub(' +', ' ', text)
            doc = nlp(text)
            matches = matcher(doc)

            regexMatches = regexMatcher(text)

            showMatches(doc, matches, regexMatches)

            for case in results.keys():
                result = inputHandling(case)
                if result == -1:
                    return results
                else:
                    results[case] += result

            clear_output()
        except:
            continue

In [168]:
def calcResults(r):
    precision = r['correct'] / (r['correct']+r['incorrect']+r['spurious'])
    recall = r['correct'] / (r['correct']+r['incorrect']+r['missing'])
    f1 = 2 * ((recall * precision)/(recall + precision))
    print(f'precision = {precision}, recall = {recall}, f1 = {f1}')

In [173]:
results = evaluate(nlp)
calcResults(results)


199
dc034afbaede3d587451c7062fd857e7_bijlage-c2-openbaar-te-maken-documenten-nationaal 78 


1056629 
 Ministerie van Financiën 
 
 do . 12/8 , 14.50 uur 
 ( _ wqbo_x ) 
 C XLea 
 TER ADVISERING 
 Aan 
 de minister 
 \ ” 
 # ‚ 
 NO Belafspraak met MSZK over delen informatie RRF met 
 sociale partners 
 notitie 
 Aanleiding 
 Op 12 augustus om 17:30u heeft u met MSZW een afspraak via Webex over zijn 
 wens om meer informatie over RRF te delen met de sociale partners . 
 Kern 
 e MEZK spreekt op 25 augustus met sociale partners over het RRF . MSZW wil 
 graag dat MEZK dan meer inhoud deelt uit de ambtelijke inventarisatie naar 
 de mogelijke invulling van een RRP die in april is gedeeld met de formatie . 
 s Wij raden u aan om tijdens het gesprek niet toe te geven aan zijn wens . U 
 kunt de volgende argumenten gebruiken : 
 o Het delen van de inhoud van de ambtelijke inventarisatie met SOPA's 
 en/of parlement is niet conform afspraak in de MR waarin tot de 
 inventarisatie besloten wer

In [174]:
results

{'correct': 172, 'incorrect': 8, 'missing': 13, 'spurious': 7}

removed all access newlines and spaces

Part of the reason not all dates were caught was an oversight with american date formats. To validate the if a found match is actually a date, it needs a date format to compare the match to. Only the dd-mm-yyyy format was checked and not the american format of mm-dd-yyyy with the month before the year. This means it excludes dates like 04-14-2022 as this cannot be a date in the dd-mm-yyyy format because there obviously isn't a 14th month. In the american system this is just april 14th 2022.

Another reason for not finding some dates is OCR mistakes. For example, in one case the date that was supposed to be found was "5/12/2021" but in the OCR process that string was read as "542/2021" where the "/" and "1" were seen as one character, a 4.

precision = 0.9197860962566845, recall = 0.8911917098445595, f1 = 0.9052631578947369

# Ministries
This next part is the evalutation of the ministries

In [3]:
def getMinisteries():
    page = requests.get('https://nl.wikipedia.org/wiki/Lijst_van_Nederlandse_ministeries')
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("td")[-1]

    results.find_all('a', href = True)
    wikis = {}

    abrr = []
    for item in str(results.find_all('p')[0]).split('\n')[:-1] + str(results.find_all('p')[1]).split('\n')[1:-1]:
        temp = re.findall('(?<=\()(.*?)(?=\))', item)
        if temp == []:
            abrr.append(None)
        elif temp[-1] == 'Nederland':
            abrr.append(None)
            if 'Overzeese Gebiedsdelen' in item:
                abrr.append(None)
        else:
            abrr.append(temp[-1].replace('&amp;', '&'))


    counter = 0
    for ministerie in results.find_all('a')[:12]:
        wikis[ministerie.text] = {'Link': 'https://nl.wikipedia.org' + ministerie['href'], 'Abbriviation' : abrr[counter]}
        counter += 1
    
    minList = list(wikis.keys()) + [wikis[x]['Abbriviation'] for x in wikis.keys()]
    for x in minList:
        temp+=x.replace(',', '').split(' ')
    
    return [x.lower() for x in temp if x != 'en' and x != '']
minList = getMinisteries()

In [60]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')

def printHilightUnderline(string):
    print('\033[4m'+string + ' ' +'\x1b[0m', end='')
    
def showMatchesMinistries(doc, minList):   
    indexOfMatches = []
    for ent in doc.ents:
        for i in range(int(ent.start), int(ent.end)):
            indexOfMatches.append(i)

    indexOfMatches = set(indexOfMatches)

    for token in doc:
        flag = False
        for mini in minList:
            if token.text.lower() == mini:
                flag = True
                break
        
        if token.i in indexOfMatches:
            printHilight(str(token.text))
            
        elif flag:
            printHilightUnderline(str(token.text))
            
        else:
            print(token, end=' ')
    
    return
    

In [61]:
def inputHandling(message):
    while(True):
        i = input(message)
        if i == 'q':
            return -1
        
        elif i == '':
            return 0
        
        try:
            i = int(i)
            return i
        
        except:
            print("input number, exit or nothing")

def evaluateMinistries(nlp, minList):
    # cor = exact match, inc = match is wrong (wrong bounds or label), mis = missing match, spu = found something that
    # isnt a mactch, 
    results = {
        'correct':0,
        'incorrect':0,
        'missing':0,
        'spurious':0,
        'partial':0
    }
    
    minList = set(minList)
    while(True):
        try:
            sample = df.sample(1)
            text = sample.text.values[0]
            text = re.sub('\n+', '\n', text)
            text = re.sub(' +', ' ', text)
            for m in minList:
                if m in text.split(' '):
                    print(m, 'AHHHHHHHHHHHHHHHH')
                    break
            else:
                continue
                
            print(sum(results.values()))
            print(sample.name.values[0], sample.page.values[0], '\n\n')
            doc = nlp(text)
            
            showMatchesMinistries(doc, minList)

            for case in results.keys():
                result = inputHandling(case)
                if result == -1:
                    return results
                else:
                    results[case] += result

            clear_output()
        except:
            continue
            
        

In [62]:
def calcResults(r):
    precision = (r['correct'] + (0.5 * r['partial'])) / (r['correct']+r['incorrect']+r['spurious'])
    recall = (r['correct'] + (0.5 * r['partial'])) / (r['correct']+r['incorrect']+r['missing'])
    f1 = 2 * ((recall * precision)/(recall + precision))
    print(f'precision = {precision}, recall = {recall}, f1 = {f1}')

In [63]:
results = evaluateMinistries(nlp_ministries, minList)
calcResults(results)

zaken AHHHHHHHHHHHHHHHH
96
6762214604a58986abf9cc852b4202e9_bijlagen-deel-7-bij-besluit-wob-verzoek-over-covid-19 59 


529309 
 
 
 | Crisiscoördinator DCC VWS | 
 Ministerie van Volksgezondheld , Welzijn en Sport | Directie Publieke Gezondheid | 
 Afdeling Crisisbeheersing en Infectieziekten | Etage : 8 flex 
 102 _ G 
 Parnassusplein 5| 2511 VX | Den Haag | 
 Postbus 20350 | 2500 EJ | Den Haag 
 
 
 
 Van : NCC - NCTV 
 Verzonden : woensdag 29 januari 2020 14:54:01 ( UTC+01:00 ) Amsterdam , Berlijn , Bern , Rome , Stockholm , Wenen 
 Onderwerp : RECTIFICATIE uitnodiging IAO maandag 3 februari om 14.00 uur met betrekking tot de stand van zaken rondom de 
 uitbraak van het Corona virus in China 
 RECTIFICATIE : 14:00 uur i.p.v. 13.30 uur 
 Geachte heer/mevrouw , 
 Op verzoek van VWS en NCTV nodig ik u uit voor een Interdepartementaal Afstemmings Overleg ( IAO ) op maandag 3 
 februari om 14.00 uur met betrekking tot de stand van zaken rondom de uitbraak van het Corona virus in China .

In [41]:
t = nlp_ministries('het Ministerie is een beetje dom')
tl = ['ministerie']
for w in t:
    print(str(w.text).lower() in tl)

False
True
False
False
False
False


In [170]:
minList

['wvc',
 'algemene',
 'zaken',
 'binnenlandse',
 'zaken',
 'koninkrijksrelaties',
 'buitenlandse',
 'zaken',
 'defensie',
 'economische',
 'zaken',
 'klimaat',
 'financiën',
 'infrastructuur',
 'waterstaat',
 'justitie',
 'veiligheid',
 'landbouw',
 'natuur',
 'voedselkwaliteit',
 'onderwijs',
 'cultuur',
 'wetenschap',
 'sociale',
 'zaken',
 'werkgelegenheid',
 'volksgezondheid',
 'welzijn',
 'sport',
 'az',
 'bzk',
 'bz',
 'def',
 'ez',
 'fin',
 'i&w',
 'j&v',
 'lnv',
 'ocw',
 'szw',
 'vws']

Missing things in situations like this: RVO/LNV. LNV should be have been caught here. Places with a lot of extra newlines or spaces within the name of a ministry will also trip up the model.

# SpaCy

This next part is the spacy analysis

In [21]:
def loadGroundTruth():
    with open('..\\data\\ner labeled data\\test.conllu', 'r', encoding='utf8') as f:
        ground = f.read()
        ground = ground.split('\n')
        ground = [x.split('\t') for x in ground]
    
    text = []
    for word in ground:
        try:
            text.append(word[1])
        except IndexError:
            text.append('\n')
    

    text = ' '.join(text)
    return text, ground


In [ ]:
text, ground = loadGroundTruth()
doc = nlp(text[:1000000])

In [88]:
def calcF1(tp, tn, fp, fn):
    print(tp, tn, fp, fn)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * ((recall * precision) / (recall + precision))
    return recall, precision, f1
    

In [112]:
def evalNER(ground, doc):
    
    groundIndex, docIndex = 0, 0
    tp, tn, fp, fn = 0, 0, 0, 0
    nerCats = ['FAC', 'PERSON', 'ORG', 'GPE', 'LOC']
    groundCats = ['ORG', 'LOC', 'PER']
    
    while True:
        try:
            if str(doc[docIndex].text) != ground[groundIndex][1]:
                
                # checks for if the words are still in sync
                if str(doc[docIndex + 1].text) != ground[groundIndex][1]:
                    docIndex += 1
                    continue
                
                elif str(doc[docIndex].text) != ground[groundIndex + 1][1]:
                    groundIndex += 1
                    continue
                    
                else:
                    for j in range(docIndex-3,docIndex+3):
                        print(doc[j], ground[j])
                
            else:
                if doc[docIndex].ent_type_ == '' and ground[groundIndex][2] == 'O':
                    tn += 1
                    
                elif doc[docIndex].ent_type_ in nerCats and ground[groundIndex][2] == 'O':
                    fp += 1
                    
                elif doc[docIndex].ent_type_ in nerCats and ground[groundIndex][2] != 'O':
                    tp += 1
                    
                elif doc[docIndex].ent_type_ == '' and ground[groundIndex][2] != 'O':
                    fn += 1
        
        except IndexError:
            pass
        
        groundIndex +=1
        docIndex +=1
        
        if groundIndex > len(ground) and docIndex > len(doc):
            print('good')
            return calcF1(tp, tn, fp, fn)

In [113]:
recall, precision, f1 = evalNER(ground, doc)
print('recall', recall)
print('precision', precision)
print('f1', f1)

good
1240 10349 47 185
recall 0.8701754385964913
precision 0.9634809634809635
f1 0.9144542772861357


In [127]:
def calcF1Strict(cor, inc, spu, mis):
    print(cor,inc,spu,mis)
    recall = cor / (cor+inc+mis)
    precision = cor / (cor+inc+spu)
    f1 = 2 * ((recall * precision) / (recall + precision))
    return recall, precision, f1

In [134]:
def evalNERStrict(ground, doc):
    
    groundIndex, docIndex = 0, 0
    cor, inc, spu, mis = 0, 0, 0, 0
    
    cats = {'ORG': ['ORG'],
           'PER': ['PERSON'],
           'LOC': ['FAC', 'GPE', 'LOC'],
           'ORG': ['ORG'],
           'MISC': ['FAC', 'PERSON', 'ORG', 'GPE', 'LOC']}
    while True:
        
        try:
            if str(doc[docIndex].text) != ground[groundIndex][1]:
                
                # checks for if the words are still in sync
                if str(doc[docIndex + 1].text) != ground[groundIndex][1]:
                    docIndex += 1
                    continue
                
                elif str(doc[docIndex].text) != ground[groundIndex + 1][1]:
                    groundIndex += 1
                    continue
                    
                else:
                    for j in range(docIndex-3,docIndex+3):
                        print(doc[j], ground[j])
                
            else:
                try:
                    # true negative
                    if doc[docIndex].ent_type_ == '' and ground[groundIndex][2] == 'O':
                        pass
                    
                    # for categories that ground doesnt have
                    elif doc[docIndex].ent_type_ not in cats['MISC'] + [''] and ground[groundIndex][2] == 'O':
                        pass

                    # spurious
                    elif doc[docIndex].ent_type_ in cats['MISC'] and ground[groundIndex][2] == 'O':
                        spu += 1

                    # missing
                    elif doc[docIndex].ent_type_ == '' and ground[groundIndex][2] != 'O':
                        mis += 1

                    # correct
                    elif doc[docIndex].ent_type_ in cats[ground[groundIndex][2][2:]]:
                        cor += 1

                    # incorrect
                    else:
                        inc += 1
                except KeyError:
                    print(doc[docIndex], ground[groundIndex])

        
        except IndexError:
            pass
        
        groundIndex +=1
        docIndex +=1
        
        if groundIndex > len(ground) and docIndex > len(doc):
            print('good')
            return calcF1Strict(cor, inc, spu, mis)

In [135]:
recall, precision, f1 = evalNERStrict(ground, doc)
print('recall', recall)
print('precision', precision)
print('f1', f1)

good
1168 261 47 185
recall 0.7236679058240396
precision 0.7913279132791328
f1 0.7559870550161812


These are really good scores for the model, especially the first one. But we need to keep in mind that the permorance of a NER model can depend on the sort of text. It could be that the spaCy model was trained on documents that have a very high resemblance to this test set. The performance on the WOB document can be lower than these results suggest